In [1]:
from baza_mip import mip_session, sessionmaker
from baza_mip.models import Procesy_Przydzielone, Procesy
from sqlalchemy import create_engine, text
from Plan_Pracy.plan_pracy import kj_paczki

import pandas as pd


gen_engine = create_engine("sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/GENERATORY.db", echo=False)



In [11]:
#dostawy
with gen_engine.begin() as conn:
    suma_paczek = conn.execute(text("select SUM(ILE_ZAMOWIONE) from ZAM_PIANKI where nr_SAMOCHODU == 'PIANPOL 18_24'")).fetchall()[0][0]
    zamowione_paczki = conn.execute(text("select NR_KOMPLETACJI, OPIS, ILE_ZAMOWIONE, ZAM1, ZAM2, NR_PARTII from ZAM_PIANKI where nr_SAMOCHODU == 'PIANPOL 18_24'")).fetchall()
    owaty = conn.execute(text("select NR_KOMPLETACJI, OPIS, ILE_ZAMOWIONE, NR_PARTII from ZAM_PIANKI where nr_SAMOCHODU == 'PIANPOL 17_24' AND MODEL == 'WILLOW'")).fetchall()
    owaty_mb = conn.execute(text("select * from OWATY")).fetchall()

In [12]:
mb_owaty = pd.DataFrame(owaty_mb)
mb_owaty

,NAZWA_UKL,OPIS,RODZAJ_CIECIA,ZUZYCIE,TYP_OWATY
0,O3-AMA-25-SP,"AMALFI 2,5",K,3.54,O3
1,O3-AMA-XLA-SP,AMALFI [LA = LA],K,4.38,O3
2,O3-AMA-XRS-SP,AMALFI [RS = RS],K,6.04,O3
3,O3-AMA-X-SP,AMALFI ][,K,0.96,O3
4,O3-AMA-15-SP,"AMALFI 3,5",K,3.54,O3
...,...,...,...,...,...
469,W3,GOYA P112x100,R,2.50,W3
470,W3,GOYA P112x80,R,2.50,W3
471,O1-OVA-X1X-SP,OVAL [1o],K,2.10,O1
472,O1-LEN-X1X-SP,LENOX [1o],K,1.90,O1


In [35]:
def dodaj_KJ(zamowione_paczki):
    czas_calkowity = 0
    kajoty = list()
    for row in zamowione_paczki:
        # print(row[0], row[1], row[2], row[3], row[4], row[5])
        
        kajoty.append([f"KJ PARTII {row[5]}: {row[1]} - {row[2]}| NR_DOS {row[3]}", kj_paczki(row[2])*10])
        
        print(f"KJ PARTII {row[5]}: {row[1]} - {row[2]}| NR_DOS {row[3]},", kj_paczki(row[2])*10)
        czas_calkowity += kj_paczki(row[2])*10
    
    print("Całkowity czas KJ:",czas_calkowity)
    return kajoty

In [14]:
def dodaj_OWATY_ROZKROJ(owaty):
    
    for row in owaty:
        # print(row[0], row[1], row[2], row[3], row[4])
        czas_ciecia = int(mb_owaty[mb_owaty.OPIS == row[1]].ZUZYCIE.sum() * 1.5)
        print(f"WYCINANIE OWAT NR PARTII {row[3]}: {row[1]} - {row[2]}szt| KOMPLETACJA {row[0]},", czas_ciecia)
 

In [28]:
def dodaj_pakowanie(zamowione_paczki):
    czas_calkowity = 0
    pakowanie = list()
    for row in zamowione_paczki:
        # print(row[0], row[1], row[2], row[3], row[4], row[5])
        
        print(f"PAKOWANIE PARTII {row[5]}: {row[1]} - {row[2]}| KOMPLETACJA {row[0]},", row[2])
        czas_calkowity += row[2]
        pakowanie.append([f"PAKOWANIE PARTII {row[5]}: {row[1]} - {row[2]}| KOMPLETACJA {row[0]}", row[2]])
    
    print("Całkowity czas PAKOWANIA:",czas_calkowity)

    return pakowanie

In [16]:
suma_paczek

620

In [17]:
zamowione_paczki

[('3_24', 'DIVA 3,5X', 40, '24/0703', '', '28/01'),
 ('3_24', 'DIVA LA', 40, '24/0703', '', '28/01'),
 ('2_24', 'LENOX [1o]', 220, '24/0703', '', '28/01'),
 ('4_24', 'OVAL [1o]', 100, '24/0703', '', '28/01'),
 ('2_24', 'KELLY [1]', 60, '24/0703', '', '28/01'),
 ('2_24', 'ONYX 3', 20, '24/0703', '', '28/01'),
 ('2_24', 'ONYX 4', 20, '24/0703', '', '28/01'),
 ('2_24', 'ONYX N', 40, '24/0703', '', '28/01'),
 ('2_24', 'ONYX ][', 80, '24/0703', '', '28/01')]

In [36]:
pakowanie = dodaj_pakowanie(zamowione_paczki)

PAKOWANIE PARTII 28/01: DIVA 3,5X - 40| KOMPLETACJA 3_24, 40
PAKOWANIE PARTII 28/01: DIVA LA - 40| KOMPLETACJA 3_24, 40
PAKOWANIE PARTII 28/01: LENOX [1o] - 220| KOMPLETACJA 2_24, 220
PAKOWANIE PARTII 28/01: OVAL [1o] - 100| KOMPLETACJA 4_24, 100
PAKOWANIE PARTII 28/01: KELLY [1] - 60| KOMPLETACJA 2_24, 60
PAKOWANIE PARTII 28/01: ONYX 3 - 20| KOMPLETACJA 2_24, 20
PAKOWANIE PARTII 28/01: ONYX 4 - 20| KOMPLETACJA 2_24, 20
PAKOWANIE PARTII 28/01: ONYX N - 40| KOMPLETACJA 2_24, 40
PAKOWANIE PARTII 28/01: ONYX ][ - 80| KOMPLETACJA 2_24, 80
Całkowity czas PAKOWANIA: 620


In [37]:
kjoty = dodaj_KJ(zamowione_paczki)

KJ PARTII 28/01: DIVA 3,5X - 40| NR_DOS 24/0703, 40
KJ PARTII 28/01: DIVA LA - 40| NR_DOS 24/0703, 40
KJ PARTII 28/01: LENOX [1o] - 220| NR_DOS 24/0703, 100
KJ PARTII 28/01: OVAL [1o] - 100| NR_DOS 24/0703, 70
KJ PARTII 28/01: KELLY [1] - 60| NR_DOS 24/0703, 50
KJ PARTII 28/01: ONYX 3 - 20| NR_DOS 24/0703, 20
KJ PARTII 28/01: ONYX 4 - 20| NR_DOS 24/0703, 20
KJ PARTII 28/01: ONYX N - 40| NR_DOS 24/0703, 40
KJ PARTII 28/01: ONYX ][ - 80| NR_DOS 24/0703, 60
Całkowity czas KJ: 440


In [38]:

kjoty

[['KJ PARTII 28/01: DIVA 3,5X - 40| NR_DOS 24/0703', 40],
 ['KJ PARTII 28/01: DIVA LA - 40| NR_DOS 24/0703', 40],
 ['KJ PARTII 28/01: LENOX [1o] - 220| NR_DOS 24/0703', 100],
 ['KJ PARTII 28/01: OVAL [1o] - 100| NR_DOS 24/0703', 70],
 ['KJ PARTII 28/01: KELLY [1] - 60| NR_DOS 24/0703', 50],
 ['KJ PARTII 28/01: ONYX 3 - 20| NR_DOS 24/0703', 20],
 ['KJ PARTII 28/01: ONYX 4 - 20| NR_DOS 24/0703', 20],
 ['KJ PARTII 28/01: ONYX N - 40| NR_DOS 24/0703', 40],
 ['KJ PARTII 28/01: ONYX ][ - 80| NR_DOS 24/0703', 60]]

In [39]:
##eliza 5
##adamr 4
##piotrl 8


mip_session.add(Procesy_Przydzielone(4, 5, 1, "DOSTAWA PARTII 28/01: NR SAM PIANPOL 18_24", "2024-08-07", 620))

for kj in kjoty:
    mip_session.add(Procesy_Przydzielone(4, 5, 2, kj[0], "2024-08-07", kj[1]))

for p in pakowanie:
    mip_session.add(Procesy_Przydzielone(4, 5, 3, p[0], "2024-08-07", p[1]))

# mip_session.add(Procesy_Przydzielone(4, 5, 11, "DOSTAWA OWAT Z:50, N:20, C:80", "2024-08-05", 150*9))

# for p in owaty:

#     mip_session.add(Procesy_Przydzielone(4, 5, 9, p[0], "2024-08-02", p[1]))


mip_session.commit()

In [41]:
mip_session.execute(text("SELECT uid, kid, proces, nazwa_procesu, data_utworzenia, planowany_dzien_rozpoczecia, priorytet, status, aktywna, uwagi_kier, preferowany_czas_wykonania FROM baza_mip.procesy_przydzielone where nazwa_procesu like '%PARTII 28/01%'")).all()

[(4, 5, 1, 'DOSTAWA PARTII 28/01: NR SAM PIANPOL 18_24', '2024-08-06', '2024-08-07', 1, 0, 1, None, 620),
 (4, 5, 1, 'DOSTAWA PARTII 28/01: NR SAM PIANPOL 18_24', '2024-08-06', '2024-08-07', 1, 0, 1, None, 620),
 (4, 5, 2, 'KJ PARTII 28/01: DIVA 3,5X - 40| NR_DOS 24/0703', '2024-08-06', '2024-08-07', 1, 0, 1, None, 40),
 (4, 5, 2, 'KJ PARTII 28/01: DIVA LA - 40| NR_DOS 24/0703', '2024-08-06', '2024-08-07', 1, 0, 1, None, 40),
 (4, 5, 2, 'KJ PARTII 28/01: LENOX [1o] - 220| NR_DOS 24/0703', '2024-08-06', '2024-08-07', 1, 0, 1, None, 100),
 (4, 5, 2, 'KJ PARTII 28/01: OVAL [1o] - 100| NR_DOS 24/0703', '2024-08-06', '2024-08-07', 1, 0, 1, None, 70),
 (4, 5, 2, 'KJ PARTII 28/01: KELLY [1] - 60| NR_DOS 24/0703', '2024-08-06', '2024-08-07', 1, 0, 1, None, 50),
 (4, 5, 2, 'KJ PARTII 28/01: ONYX 3 - 20| NR_DOS 24/0703', '2024-08-06', '2024-08-07', 1, 0, 1, None, 20),
 (4, 5, 2, 'KJ PARTII 28/01: ONYX 4 - 20| NR_DOS 24/0703', '2024-08-06', '2024-08-07', 1, 0, 1, None, 20),
 (4, 5, 2, 'KJ PARTII 